In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv
import os
import pandas as pd
import numpy as np
import random
import torch.nn.init as init
import pdb
from sklearn.metrics import mean_absolute_percentage_error
torch.autograd.set_detect_anomaly(True)
seed = 42

def file_name(file_dir,file_type='.csv'):#默认为文件夹下的所有文件
    lst = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if(file_type == ''):
                lst.append(file)
            else:
                if os.path.splitext(file)[1] == str(file_type):#获取指定类型的文件名
                    lst.append(file)
    return lst

def normalize0(inputs):
    normalized = []
    for eq in inputs:
        maks = np.max(np.abs(eq))
        if maks != 0:
            normalized.append(eq / maks)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize1(inputs):
    normalized = []
    for eq in inputs:
        mean = np.mean(eq)
        std = np.std(eq)
        if std != 0:
            normalized.append((eq - mean) / std)
        else:
            normalized.append(eq)
    return np.array(normalized)


def normalize(inputs):
    normalized = []
    for eq in inputs:
        with np.errstate(invalid='ignore'):
            eps = 1e-10  # 可以根据需要调整epsilon的值

            eq_log = [np.log(x + eps) if i < 5 else x for i, x in enumerate(eq)]

            #eq_log = [np.log(x) if i < 5 else x for i, x in enumerate(eq)]
            eq_log1 = np.nan_to_num(eq_log).tolist()
            normalized.append(eq_log1)
    return np.array(normalized)


def k_fold_split(inputs, targets, K, seed=None):
    # 确保所有随机操作都使用相同的种子
    if seed is not None:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

    ind = int(len(inputs) / K)
    inputsK = []
    targetsK = []

    for i in range(0, K - 1):
        inputsK.append(inputs[i * ind:(i + 1) * ind])
        targetsK.append(targets[i * ind:(i + 1) * ind])

    inputsK.append(inputs[(i + 1) * ind:])
    targetsK.append(targets[(i + 1) * ind:])

    return inputsK, targetsK


def merge_splits(inputs, targets, k, K):
    if k != 0:
        z = 0
        inputsTrain = inputs[z]
        targetsTrain = targets[z]
    else:
        z = 1
        inputsTrain = inputs[z]
        targetsTrain = targets[z]

    for i in range(z + 1, K):
        if i != k:
            inputsTrain = np.concatenate((inputsTrain, inputs[i]))
            targetsTrain = np.concatenate((targetsTrain, targets[i]))

    return inputsTrain, targetsTrain, inputs[k], targets[k]


def targets_to_list(targets):
    targetList = np.array(targets)

    return targetList


In [21]:
from torch import nn
import torch
import torch.nn.functional as F
class nconv(nn.Module):
    def __init__(self):
        super(nconv, self).__init__()

    def forward(self, x, A):
        x = torch.einsum('ncvl,vw->ncwl', (x, A))
        return x.contiguous()

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903 
    图注意力层
    input: (B,N,C_in)
    output: (B,N,C_out)
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.in_features = in_features   # 节点表示向量的输入特征数
        self.out_features = out_features   # 节点表示向量的输出特征数
        self.dropout = dropout    # dropout参数
        self.alpha = alpha     # leakyrelu激活的参数
        self.concat = concat   # 如果为true, 再进行elu激活
        
        # 定义可训练参数，即论文中的W和a
        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))  
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # 初始化
        self.A = nn.Parameter(torch.zeros(size=(2*out_features, 16)))
        nn.init.xavier_uniform_(self.A.data, gain=1.414)   # 初始化
        
        # 定义leakyrelu激活函数
        self.leakyrelu = nn.LeakyReLU(self.alpha)
    
    def forward(self, inp, adj):
        """
        inp: input_fea [B,N, in_features]  in_features表示节点的输入特征向量元素个数
        adj: 图的邻接矩阵  [N, N] 非零即一，数据结构基本知识
        """
        h = torch.matmul(inp.double(), self.W.double())   # [B, N, out_features]
        N = h.size()[1]    # N 图的节点数

        a_input = torch.cat([h.repeat(1,1,N).view(-1, N*N, self.out_features), h.repeat(1, N, 1)], dim=-1).view(-1, N, N, 2*self.out_features)
        # [B, N, N, 2*out_features]
      
        E = []
        for i in range(16):
            a = self.A[:,i].unsqueeze(1)
            e_col = torch.matmul(a_input.double(), a.double()).squeeze(3)[:,:,i]
            E.append(e_col)

        e = self.leakyrelu(torch.stack(E, dim=2))
        # [B, N, N, 1] => [B, N, N] 图注意力的相关系数（未归一化）
        
        zero_vec = -1e12 * torch.ones_like(e)    # 将没有连接的边置为负无穷


        attention = torch.where(adj>0, e, zero_vec)   # [B, N, N]
        # 表示如果邻接矩阵元素大于0时，则两个节点有连接，该位置的注意力系数保留，
        # 否则需要mask并置为非常小的值，原因是softmax的时候这个最小值会不考虑。
        attention = F.softmax(attention, dim=1)    # softmax形状保持不变 [B, N, N]，得到归一化的注意力权重！
        attention = F.dropout(attention, self.dropout, training=self.training)   # dropout，防止过拟合
        h_prime = torch.matmul(attention, h)  # [B, N, N].[B, N, out_features] => [B, N, out_features]
        # 得到由周围节点通过注意力权重进行更新的表示
        if self.concat:
            return F.relu(h_prime)
        else:
            return h_prime 
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GAT(nn.Module):
    def __init__(self, n_feat, n_hid, n_class, dropout, alpha, n_heads):
        """Dense version of GAT
        n_heads 表示有几个GAL层，最后进行拼接在一起，类似self-attention
        从不同的子空间进行抽取特征。
        """
        super(GAT, self).__init__()
        self.dropout = dropout 
        self.leakyrelu = nn.LeakyReLU()
        
        # 定义multi-head的图注意力层
        self.attentions = [GraphAttentionLayer(n_feat, n_hid, dropout=dropout, alpha=alpha, concat=True) for _ in range(n_heads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)   # 加入pytorch的Module模块
        # 输出层，也通过图注意力层来实现，可实现分类、预测等功能
        self.out_att = GraphAttentionLayer(n_hid * n_heads, n_class, dropout=dropout,alpha=alpha, concat=False)
    
    
    def test(self, test_loader):
        self.eval()
        predictions = []
        for test_inputs, test_graph_input, _ in test_loader:
            batch_predictions = self(test_inputs, test_graph_input)
            predictions.append(batch_predictions)
        predictions = torch.cat(predictions)
        return predictions

    
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)   # dropout，防止过拟合
        x = torch.cat([att(x, adj) for att in self.attentions], dim=2)  # 将每个head得到的表示进行拼接
        x = F.dropout(x, self.dropout, training=self.training)   # dropout，防止过拟合
        x = self.out_att(x, adj)   # 输出并激活
        #x = F.log_softmax(x, dim=2)[:, -1, :]
        # print(x)
        # print(x)
        # print(x.shape)
        return x[:, -1, :] # log_softmax速度变快，保持数值稳定


    def fit(self,train_loader, val_loader,num_epochs=5000, patience=100):
        #best_val_loss = float('inf')
        best_val_loss = torch.tensor(float('inf'), dtype=torch.double)
        patience_counter = 0
        optimizer = optim.Adam(self.parameters(), lr=0.0015,weight_decay=5e-4)
        criterion = nn.MSELoss()
        # 使用平滑的 L1 损失，也称为 Huber loss
        # criterion = nn.SmoothL1Loss()

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0 
            for inputs, graph_input, targets in train_loader:
                optimizer.zero_grad()
                outputs = self(inputs, graph_input)
                loss = criterion(outputs.squeeze(dim=1), targets)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm = 1)
                optimizer.step()
            #     train_loss += loss.item()
            # avg_train_loss = train_loss / len(train_loader)
            

            self.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_graph_input, val_targets in val_loader:
                    val_outputs = self(val_inputs, val_graph_input)
                    val_loss = criterion(val_outputs.squeeze(dim=1), val_targets)
            #         val_loss += val_loss.item()
            # avg_val_loss=val_loss / len(val_loader)
            if (epoch + 1) % 20 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')
                

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f'Early stopping at epoch {epoch+1}')
                    return

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming your PyTorch model is defined as before


if __name__ == "__main__":
    # stock_info = sys.argv[0]
    # lag_bin = int(sys.argv[1])
    # lag_day = int(sys.argv[2])
    # bin_num = int(sys.argv[3])
    # random_state_here = int(sys.argv[4])
    # test_set_size = float(sys.argv[5])
    lag_bin = 3
    lag_day = 3
    num_nodes = (int(lag_bin)+1)*(int(lag_day)+1)
    forecast_days = 15
    bin_num=24
    random_state_here = 88
    mape_list = []
    data_dir = './data/volume/0308/'
    files =file_name('./data/')
    stocks_info = list(set(s.split('_25')[0] for s in files))
    print(stocks_info)
    for stock_info in stocks_info[0:2]:
        print(f'>>>>>>>>>>>>>>>>>>>>{stock_info}>>>>>>>>>>>>>>>>>>>>>>>')
        data_dir1 = f'{data_dir}{stock_info}_{lag_bin}_{lag_day}'
        test_set_size = bin_num*forecast_days
        K = 5
        inputs_data = np.load(f'{data_dir1}_inputs.npy', allow_pickle=True)
        inputs_data = [[[torch.tensor(x, dtype=torch.float64) for x in sublist] for sublist in list1] for list1 in inputs_data]
        array_data = np.array(inputs_data)
        inputs = np.reshape(array_data, (len(inputs_data), num_nodes,-1))
        targets = np.load(f'{data_dir1}_output.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.load(f'{data_dir1}_graph_input.npy', allow_pickle=True).astype(np.float64)
        graph_input = np.array([graph_input] * inputs.shape[0])
        graph_features = np.load(f'{data_dir1}_graph_coords.npy', allow_pickle=True).astype(np.float64)
        graph_features = np.array([graph_features] * inputs.shape[0])

        trainInputs, testInputs, traingraphInput, testgraphInput, traingraphFeature, testgraphFeature, trainTargets, testTargets = train_test_split(inputs, graph_input, graph_features, targets, test_size=test_set_size, 
                                                     random_state=random_state_here)
        testInputs = normalize(testInputs)
        # testInputs = test_inputs
        inputsK, targetsK = k_fold_split(trainInputs, trainTargets, K)

        mape_list = []

        test_dataset = TensorDataset(torch.tensor(testInputs), torch.tensor(testgraphInput), torch.tensor(testTargets))
        test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)
        K = 5  # Number of folds
        for k in range(K):
            torch.manual_seed(0)  # Set a random seed for reproducibility

            trainInputsAll, trainTargets, valInputsAll, valTargets = merge_splits(inputsK, targetsK, k, K)

            trainGraphInput = traingraphInput[0:trainInputsAll.shape[0], :]
            trainGraphFeatureInput = traingraphFeature[0:trainInputsAll.shape[0], :]

            valGraphInput = traingraphInput[0:valInputsAll.shape[0], :]
            valGraphFeatureInput = traingraphFeature[0:valInputsAll.shape[0], :]

            trainInputs = normalize(trainInputsAll[:, :])
            valInputs = normalize(valInputsAll[:, :])

            # Assuming trainInputs, trainGraphInput, trainGraphFeatureInput, trainTargets are PyTorch tensors
            train_dataset = TensorDataset(torch.tensor(trainInputs), torch.tensor(trainGraphInput),torch.tensor(trainTargets))
            val_dataset = TensorDataset(torch.tensor(valInputs), torch.tensor(valGraphInput),torch.tensor(valTargets))

            # train_dataset = TensorDataset(torch.tensor(trainInputs, dtype=torch.float32), torch.tensor(trainGraphInput, dtype=torch.float32), torch.tensor(trainGraphFeatureInput, dtype=torch.float32), torch.tensor(trainTargets, dtype=torch.float32))
            # val_dataset = TensorDataset(torch.tensor(valInputs, dtype=torch.float32), torch.tensor(valGraphInput, dtype=torch.float32), torch.tensor(valGraphFeatureInput, dtype=torch.float32), torch.tensor(valTargets, dtype=torch.float32))

            train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False)

        
            model = GAT(7,8,1,0.6,0.3,8)
            model.fit(train_loader, val_loader)
            predictions=model.test(test_loader)
            torch.save(model.state_dict(), f'models/gat_{stock_info}_{lag_bin}_{lag_day}_gcn_model_iteration_{k}.pt')
    
            print()
            print('Fold number:', k)

            new_predictions = np.array([item.detach().numpy() for item in predictions]).flatten()
            MAPE = []

            MAPE.append(mean_absolute_percentage_error(testTargets[:], new_predictions[:]))
            print(new_predictions)
            print(MAPE)
            testTargets0 = list(testTargets)

            res = {
                'testTargets': testTargets0,
                'new_predictions': new_predictions
            }

            res_df = pd.DataFrame(res)
            res_df.to_csv(f'./result/gat_{stock_info}_{lag_bin}_{lag_day}_res_test_MAPE{k}.csv', index=False)

            print('MAPE = ', np.array(MAPE).mean())
            MAPE_mean = np.array(MAPE).mean()
            mape_list.append(MAPE)

        print('-')
        print('mape score = ', mape_list)



['300540_XSHE', '000046_XSHE', '300174_XSHE', '300133_XSHE', '603053_XSHG', '300263_XSHE', '603095_XSHG', '300343_XSHE', '002679_XSHE', '002282_XSHE', '000951_XSHE', '000998_XSHE', '000753_XSHE', '002841_XSHE', '300433_XSHE', '603359_XSHG', '600622_XSHG', '002882_XSHE']
>>>>>>>>>>>>>>>>>>>>300540_XSHE>>>>>>>>>>>>>>>>>>>>>>>
Epoch [20/5000], Train Loss: 127437829441.1225, Val Loss: 29878993345.2005
Epoch [40/5000], Train Loss: 49641341230.7724, Val Loss: 22733513988.8577
Epoch [60/5000], Train Loss: 47546555643.5168, Val Loss: 30620584253.5149
Epoch [80/5000], Train Loss: 69164987459.2881, Val Loss: 32354738431.1619
Epoch [100/5000], Train Loss: 19970733912.3916, Val Loss: 31732936820.7914
Early stopping at epoch 111

Fold number: 0
[7.80539094e+02 8.15184914e+02 7.59182276e+03 2.91767242e+04
 6.24601393e+04 1.66352434e+04 5.85265285e+04 1.95418765e+03
 5.61889757e+02 2.79617019e+04 3.53624656e+04 4.42072233e+04
 2.64723111e+03 2.48655294e+02 8.97986506e+02 4.60130191e+03
 6.83610274e+0

KeyboardInterrupt: 